In [ ]:
all_classes = ["Clover", "Grass", "Dung", "Sheep Dung", "Soil", "Artifact"]

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import json

# Load the dataset
df = pd.read_csv("/content/dataset-v4-raw.csv")

print(df.shape)

# Define the image directory
image_dir = '/content/content/content/flat_split'  # Change this to the directory where images are extracted

# Parse the 'choice' column to extract labels
def parse_choices(choice_str):
    if pd.isna(choice_str) or not choice_str.strip() or len(choice_str) == 0:
        return None
    try:
        choices = json.loads(choice_str.replace("'", "\""))['choices'] if choice_str[0] == '{' else [choice_str]
        # label_map = {"Clover": 0, "Grass": 1, "Dung": 2, "Sheep Dung": 3, "Soil": 4, "Artifact": 5}

        # label = [0 for _ in range(len(label_map))]
        # for choice in choices:
        #   label[label_map[choice]] = 1
        # return label
        return choices
    except (json.JSONDecodeError, KeyError, TypeError):
        print(choice_str)
        return None

df['labels'] = df['choice'].apply(parse_choices)

print(df.shape)

# Filter out rows with parsing errors (None) but keep rows with empty lists
df = df[df['labels'].notna()]

print(df.shape)

# Create a function to map the image path
def get_full_path(img_name):
    return img_name.split('/')[-1].replace("%20", " ")

df['image'] = df['image'].apply(get_full_path)

# Split the data into training and validation sets
train_df, val_df = train_test_split(df, train_size=0.8, random_state=42)

(18560, 8)
(18560, 9)
(18491, 9)


<ipython-input-46-5de99155a78d>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image'] = df['image'].apply(get_full_path)


In [ ]:
df

,annotation_id,annotator,choice,created_at,id,image,lead_time,updated_at,labels
0,313,4,Grass,2024-07-11T10:12:24.449200Z,18194,Bare Soil_Fran_0_194_388.jpg,4.744,2024-07-11T10:12:24.449230Z,[Grass]
1,376,4,"{""choices"":[""Grass"",""Soil""]}",2024-07-11T10:15:08.588492Z,18226,Bare Soil_Fran_0_582_776.jpg,3.141,2024-07-11T10:15:08.588532Z,"[Grass, Soil]"
2,385,4,"{""choices"":[""Grass"",""Soil""]}",2024-07-11T10:15:31.489451Z,18230,Bare Soil_Fran_0_776_388.jpg,3.113,2024-07-11T10:15:31.489485Z,"[Grass, Soil]"
3,416,4,Grass,2024-07-11T10:17:14.445165Z,18242,Bare Soil_Fran_1_0_388.jpg,9.326,2024-07-11T10:17:14.445195Z,[Grass]
4,425,4,"{""choices"":[""Grass"",""Soil""]}",2024-07-11T10:17:41.455977Z,18246,Bare Soil_Fran_1_1164_0.jpg,4.419,2024-07-11T10:17:41.456008Z,"[Grass, Soil]"
...,...,...,...,...,...,...,...,...,...
18110,8584,9,"{""choices"":[""Clover"",""Grass""]}",2024-07-17T12:32:18.853288Z,114424,More Clover_7_194_1552.jpg,2.547,2024-07-17T12:32:18.853306Z,"[Clover, Grass]"
18111,8537,9,Grass,2024-07-17T12:31:26.238375Z,114439,More Clover_7_388_388.jpg,5.071,2024-07-17T12:31:26.238392Z,[Grass]
18112,8535,10,Grass,2024-07-17T12:31:24.136156Z,114439,More Clover_7_388_388.jpg,2.202,2024-07-17T12:31:24.136174Z,[Grass]
18113,8313,5,"{""choices"":[""Clover"",""Grass""]}",2024-07-17T12:19:27.506188Z,114568,More Clover_9_0_1746.jpg,1.795,2024-07-17T12:19:27.506207Z,"[Clover, Grass]"


In [ ]:
# Count images for each class
count_grass = 0
count_not_grass = 0
for labels in df['labels']:
  if len(labels) == 1 and labels[0] == 'Grass':
    count_grass += 1
  if not "Grass" in labels:
    count_not_grass += 1

print(count_grass)
print(count_not_grass)


for class_name in all_classes:
  count = 0
  for labels in df['labels']:
    if class_name in labels:
      count += 1
  print(f"Number of images labelled as {class_name}: {count}")

11872
972
Number of images labelled as Clover: 3395
Number of images labelled as Grass: 17519
Number of images labelled as Dung: 1023
Number of images labelled as Sheep Dung: 220
Number of images labelled as Soil: 1613
Number of images labelled as Artifact: 738


In [ ]:
df.to_csv('dataset-v4-clean.csv')

In [ ]:
# Remove entries with the label "Artifact"
df = df[~df['labels'].apply(lambda x: 'Artifact' in x)]

In [ ]:
def analyse_df(df):
  # Count images for each class
  count_grass = 0
  count_not_grass = 0
  for labels in df['labels']:
    if len(labels) == 1 and labels[0] == 'Grass':
      count_grass += 1
    if not "Grass" in labels:
      count_not_grass += 1

  print(count_grass)
  print(count_not_grass)


  for class_name in all_classes:
    count = 0
    for labels in df['labels']:
      if class_name in labels:
        count += 1
    print(f"Number of images labelled as {class_name}: {count}")

In [ ]:
analyse_df(df)

11872
816
Number of images labelled as Clover: 3365
Number of images labelled as Grass: 16937
Number of images labelled as Dung: 1014
Number of images labelled as Sheep Dung: 219
Number of images labelled as Soil: 1544
Number of images labelled as Artifact: 0


In [ ]:
# Remove entries with the label "Sheep Dung"
df = df[~df['labels'].apply(lambda x: 'Sheep Dung' in x)]

In [ ]:
analyse_df(df)

11872
769
Number of images labelled as Clover: 3360
Number of images labelled as Grass: 16765
Number of images labelled as Dung: 1004
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 1435
Number of images labelled as Artifact: 0


In [ ]:
def sample_rows(df, predicate, num_samples):
    """
    Sample multiple rows from the dataframe matching the predicate, add them to a new dataset,
    and delete them from the original dataset.

    Parameters:
    df (pd.DataFrame): The original dataframe.
    predicate (function): A function that takes a row and returns True if the row matches the condition.
    num_samples (int): The number of rows to sample.

    Returns:
    pd.DataFrame, pd.DataFrame: The updated original dataframe and the new dataset with the sampled rows.
    """
    # Filter the rows matching the predicate
    matching_rows = df[df.apply(predicate, axis=1)]

    # If no rows match the predicate, return the original dataframe and an empty new dataframe
    if matching_rows.empty:
        return df, pd.DataFrame(columns=df.columns)

    # Randomly sample the specified number of rows from the matching rows
    sampled_rows = matching_rows.sample(n=num_samples)

    # Remove the sampled rows from the original dataframe
    df_updated = df.drop(sampled_rows.index)

    # Create a new dataframe with the sampled rows
    new_df = sampled_rows

    return df_updated, new_df

In [ ]:
# Include all non-grass

predicate = lambda row: 'Grass' not in row['labels']

# Calculate the number of rows matching the predicate
num_samples = df['labels'].apply(lambda x: 'Grass' not in x).sum()

# Sample rows from the original dataframe
df_updated, new_dataset = sample_rows(df, predicate, num_samples)

In [ ]:
analyse_df(df_updated)

11872
0
Number of images labelled as Clover: 3105
Number of images labelled as Grass: 16765
Number of images labelled as Dung: 956
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 957
Number of images labelled as Artifact: 0


In [ ]:
analyse_df(new_dataset)

0
769
Number of images labelled as Clover: 255
Number of images labelled as Grass: 0
Number of images labelled as Dung: 48
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 478
Number of images labelled as Artifact: 0


In [ ]:
df = df_updated

In [ ]:
analyse_df(df)

11872
0
Number of images labelled as Clover: 3105
Number of images labelled as Grass: 16765
Number of images labelled as Dung: 956
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 957
Number of images labelled as Artifact: 0


In [ ]:
# Include all Dung

predicate = lambda row: 'Dung' in row['labels']

# Calculate the number of rows matching the predicate
num_samples = df['labels'].apply(lambda x: 'Dung' in x).sum()

# Sample rows from the original dataframe
df_updated, new_dataset2 = sample_rows(df, predicate, num_samples)

analyse_df(df_updated)
analyse_df(new_dataset2)

11872
0
Number of images labelled as Clover: 3053
Number of images labelled as Grass: 15809
Number of images labelled as Dung: 0
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 924
Number of images labelled as Artifact: 0
0
0
Number of images labelled as Clover: 52
Number of images labelled as Grass: 956
Number of images labelled as Dung: 956
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 33
Number of images labelled as Artifact: 0


In [ ]:
new_dataset = pd.concat([new_dataset, new_dataset2], ignore_index=True)

In [ ]:
analyse_df(new_dataset)

0
769
Number of images labelled as Clover: 307
Number of images labelled as Grass: 956
Number of images labelled as Dung: 1004
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 511
Number of images labelled as Artifact: 0


In [ ]:
df = df_updated

In [ ]:
analyse_df(df)

11872
0
Number of images labelled as Clover: 3053
Number of images labelled as Grass: 15809
Number of images labelled as Dung: 0
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 924
Number of images labelled as Artifact: 0


In [ ]:
# Add 1004 - 511 soil

predicate = lambda row: 'Soil' in row['labels']

# Sample rows from the original dataframe
df_updated, new_dataset2 = sample_rows(df, predicate, 1004 - 511)

analyse_df(df_updated)
analyse_df(new_dataset2)


11872
0
Number of images labelled as Clover: 3029
Number of images labelled as Grass: 15316
Number of images labelled as Dung: 0
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 431
Number of images labelled as Artifact: 0
0
0
Number of images labelled as Clover: 24
Number of images labelled as Grass: 493
Number of images labelled as Dung: 0
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 493
Number of images labelled as Artifact: 0


In [ ]:
new_dataset = pd.concat([new_dataset, new_dataset2], ignore_index=True)

In [ ]:
df = df_updated

In [ ]:
analyse_df(new_dataset)
analyse_df(df)

0
769
Number of images labelled as Clover: 331
Number of images labelled as Grass: 1449
Number of images labelled as Dung: 1004
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 1004
Number of images labelled as Artifact: 0
11872
0
Number of images labelled as Clover: 3029
Number of images labelled as Grass: 15316
Number of images labelled as Dung: 0
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 431
Number of images labelled as Artifact: 0


In [ ]:
# Add 1004 - 331 clover

predicate = lambda row: 'Clover' in row['labels']

# Sample rows from the original dataframe
df_updated, new_dataset2 = sample_rows(df, predicate, 1004 - 331)

analyse_df(df_updated)
analyse_df(new_dataset2)


11872
0
Number of images labelled as Clover: 2356
Number of images labelled as Grass: 14643
Number of images labelled as Dung: 0
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 428
Number of images labelled as Artifact: 0
0
0
Number of images labelled as Clover: 673
Number of images labelled as Grass: 673
Number of images labelled as Dung: 0
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 3
Number of images labelled as Artifact: 0


In [ ]:
new_dataset = pd.concat([new_dataset, new_dataset2], ignore_index=True)
df = df_updated

In [ ]:
analyse_df(new_dataset)
analyse_df(df)

0
769
Number of images labelled as Clover: 1004
Number of images labelled as Grass: 2122
Number of images labelled as Dung: 1004
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 1007
Number of images labelled as Artifact: 0
11872
0
Number of images labelled as Clover: 2356
Number of images labelled as Grass: 14643
Number of images labelled as Dung: 0
Number of images labelled as Sheep Dung: 0
Number of images labelled as Soil: 428
Number of images labelled as Artifact: 0


In [ ]:
new_dataset.to_csv('dataset-v4-clean-resampled.csv')

In [ ]:
all_classes = ["Clover", "Grass", "Dung", "Soil"]

In [ ]:
import pandas as pd
import numpy as np

def split_train_val(df, class_label, train_ratio=0.8):
    """
    Split the dataset into training and validation sets for a given class label.

    Parameters:
    df (pd.DataFrame): The original dataframe.
    class_label (str): The class label to split by.
    train_ratio (float): The ratio of the training set.

    Returns:
    pd.DataFrame: The dataframe with the 'subset' column updated.
    """
    # Filter rows that contain the class label
    class_rows = df[df['labels'].apply(lambda x: class_label in x)]

    # Determine the number of training samples
    num_train_samples = int(len(class_rows) * train_ratio)

    # Shuffle the rows
    shuffled_rows = class_rows.sample(frac=1, random_state=42)

    # Split into training and validation sets
    train_rows = shuffled_rows.iloc[:num_train_samples]
    val_rows = shuffled_rows.iloc[num_train_samples:]

    # Update the 'subset' column
    df.loc[train_rows.index, 'subset'] = 'Train'
    df.loc[val_rows.index, 'subset'] = 'Val'

    return df

# Assume new_dataset is already loaded
new_dataset['subset'] = np.nan

# Split dataset for each class
for class_label in all_classes:
    new_dataset = split_train_val(new_dataset, class_label)

# Save the result
new_dataset_split = new_dataset

In [ ]:
new_dataset_split = new_dataset_split.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
new_dataset_split.to_csv('dataset-v4-clean-resampled-split.csv')

In [ ]:
# Verify
cleaned_df = pd.read_csv('dataset-v4-clean-resampled-split.csv')

# Print the number of training and validation rows for each class
for class_label in all_classes:
    num_train = len(cleaned_df[(new_dataset_split['labels'].apply(lambda x: class_label in x)) & (cleaned_df['subset'] == 'Train')])
    num_val = len(cleaned_df[(new_dataset_split['labels'].apply(lambda x: class_label in x)) & (cleaned_df['subset'] == 'Val')])
    print(f"Number of training images labelled as {class_label}: {num_train}")
    print(f"Number of validation images labelled as {class_label}: {num_val}")
    print(f"Proportion training {num_train / (num_train + num_val)}\n")

Number of training images labelled as Clover: 782
Number of validation images labelled as Clover: 222
Proportion training 0.7788844621513944

Number of training images labelled as Grass: 1687
Number of validation images labelled as Grass: 435
Proportion training 0.795004712535344

Number of training images labelled as Dung: 803
Number of validation images labelled as Dung: 201
Proportion training 0.799800796812749

Number of training images labelled as Soil: 805
Number of validation images labelled as Soil: 202
Proportion training 0.7994041708043694

